In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold 
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

In [2]:
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)

In [3]:
data

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11.0,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10.0,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5

In [4]:
#erase last remaining NaNs
data = data.drop([83217,93811], axis=0).reset_index()

In [5]:
#x-y split
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(np.object)

# we OneHotEncode the categoricals so we can use the same dataset to perform a regression later (in the lab).
# it is not needed for a DecisionTree or RandomForest model
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([numericalX, encoded_categorical], axis = 1)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [6]:
# we dont need numerical normalization/scaling for random forest

In [7]:
# for downsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)
# quicker way to downsample category 0:
category_0 = trainset[trainset['TARGET_B']==0].sample(len(trainset[trainset['TARGET_B']==1]))
print(category_0.shape)

category_1 = trainset[trainset['TARGET_B']== 1 ]
trainset_new = pd.concat([category_0, category_1], axis = 0)

#### this is not randomized anymore, because no category 0 is on top and catgeory 1 on bottom
### need to randomize those rows again######

trainset_new = trainset_new.sample(frac =1) #randomize the rows
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']
#data = data.reset_index(drop=True)
print(X_train.shape)

(3832, 357)
(7664, 356)


In [8]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

#need this later, but not now
y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

# Now we can remove the column target d from the set of features
# we need another target that we dont look at now
X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

In [9]:
X_train

,index,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
78204,78204,2,64.000000,1,9,0,2,40,36,36,1,3,3,8,799,228,379,99,0,0,47,53,97,1,1,2,2,0,0,0,1,0,0,0,1,0,0,1,37,47,54,42,50,54,19,46,33,21,13,20,14,9,10,15,18,18,18,34,16,14,39,18,39,34,41,25,13,4,1,56,15,9,20,140,210,32,32,0,37,37,33,0,0,0,621,626,4,3,57,43,96,4,18,23,60,48,15,76,24,8,2,6,16,28,6,15,50,32,0,0,0,0,0,1,12,83,0,6,31,47,49,12,4,26,11,30,4,4,34,1,2,7,74,77,1080.0,445.0,757.0,244,383,313,404,15640,25,26,11,19,14,3,0,0,1,7,26,11,27,20,6,0,0,2,49,3,39,5,54,46,19,75,18,2,2,0,0,0,2,4,0,2,14,16,0,50,58,70,47,65,47,85,58,99,99,0,9,22,2,11,19,0,3,18,0,7,7,1,1,4,0,6,13,9,0,12,15,16,4,1,0,12,1,5,3,1,3,3,1,82,9,0,128,4,13,28,27,11,14,3,3,11,1,8,5,1,2,0,19,40,0,36,13,36,23,2,9,1,7,0,0,4,0,2,0,0,0,1,1,0,2,35,92,2,2,3,91,54,3,4,18,3,13,24,61,93,7,0,0,41,3,55,1,0,0,97,3,98,99,99,5,2,2,9,25,64,6,12,111.0,23,12,3.0,7.0,4.0,5,4.826087,133647,1,4,13,26,2,2,90,1,34,1,95,2,92,12,95,12,90,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
55531,55531,1,82.000000,4,9,0,0,28,29,43,4,2,2,9,1253,377,471,99,0,0,49,51,94,5,0,0,2,0,0,0,0,0,0,0,1,0,0,1,38,45,47,39,48,51,22,41,40,20,9,17,22,16,15,13,9,17,18,29,22,14,31,10,29,18,39,43,24,11,4,66,7,7,20,182,265,97,96,3,3,0,0,0,0,0,1139,1223,6,5,96,4,98,2,0,32,80,69,28,91,9,4,1,3,10,18,2,18,55,21,0,3,1,8,21,64,86,94,1,0,88,66,69,14,4,3,0,0,2,2,0,0,61,78,89,94,2160.0,57.0,505.0,630,676,670,713,27594,4,4,12,17,25,22,9,1,6,2,1,12,15,27,25,12,0,5,23,0,80,0,26,74,4,93,6,0,0,0,0,0,0,1,0,7,16,19,1,53,68,82,55,82,54,35,20,57,0,4,32,19,8,14,13,0,0,2,0,8,0,0,3,0,0,2,29,5,5,6,8,5,5,2,2,12,7,10,2,4,2,2,4,71,17,0,154,2,3,16,25,8,32,14,12,21,3,16,14,0,0,0,15,28,1,29,16,43,2,1,4,0,10,0,2,7,9,0,3,0,0,0,0,0,3,72,92,0,0,8,99,68,20,13,44,0,2,3,4,8,92,0,0,97,0,2,1,0,0,99,0,99,99,99,10,4,10,6,23,59,5,10,106.0,6,4,10.0,20.0,20.0,9,17.666667,66224,1,1,2,13,2,1,89,1,16,2,89,3,91,10,95,4,89,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
59597,59597,0,36.000000,2,9,0,0,24,52,15,4,0,1,3,929,243,372,99,0,0

In [10]:
#drop index column
X_train.drop('index', axis=1, inplace=True)

In [11]:
X_train

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
78204,2,64.000000,1,9,0,2,40,36,36,1,3,3,8,799,228,379,99,0,0,47,53,97,1,1,2,2,0,0,0,1,0,0,0,1,0,0,1,37,47,54,42,50,54,19,46,33,21,13,20,14,9,10,15,18,18,18,34,16,14,39,18,39,34,41,25,13,4,1,56,15,9,20,140,210,32,32,0,37,37,33,0,0,0,621,626,4,3,57,43,96,4,18,23,60,48,15,76,24,8,2,6,16,28,6,15,50,32,0,0,0,0,0,1,12,83,0,6,31,47,49,12,4,26,11,30,4,4,34,1,2,7,74,77,1080.0,445.0,757.0,244,383,313,404,15640,25,26,11,19,14,3,0,0,1,7,26,11,27,20,6,0,0,2,49,3,39,5,54,46,19,75,18,2,2,0,0,0,2,4,0,2,14,16,0,50,58,70,47,65,47,85,58,99,99,0,9,22,2,11,19,0,3,18,0,7,7,1,1,4,0,6,13,9,0,12,15,16,4,1,0,12,1,5,3,1,3,3,1,82,9,0,128,4,13,28,27,11,14,3,3,11,1,8,5,1,2,0,19,40,0,36,13,36,23,2,9,1,7,0,0,4,0,2,0,0,0,1,1,0,2,35,92,2,2,3,91,54,3,4,18,3,13,24,61,93,7,0,0,41,3,55,1,0,0,97,3,98,99,99,5,2,2,9,25,64,6,12,111.0,23,12,3.0,7.0,4.0,5,4.826087,133647,1,4,13,26,2,2,90,1,34,1,95,2,92,12,95,12,90,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
55531,1,82.000000,4,9,0,0,28,29,43,4,2,2,9,1253,377,471,99,0,0,49,51,94,5,0,0,2,0,0,0,0,0,0,0,1,0,0,1,38,45,47,39,48,51,22,41,40,20,9,17,22,16,15,13,9,17,18,29,22,14,31,10,29,18,39,43,24,11,4,66,7,7,20,182,265,97,96,3,3,0,0,0,0,0,1139,1223,6,5,96,4,98,2,0,32,80,69,28,91,9,4,1,3,10,18,2,18,55,21,0,3,1,8,21,64,86,94,1,0,88,66,69,14,4,3,0,0,2,2,0,0,61,78,89,94,2160.0,57.0,505.0,630,676,670,713,27594,4,4,12,17,25,22,9,1,6,2,1,12,15,27,25,12,0,5,23,0,80,0,26,74,4,93,6,0,0,0,0,0,0,1,0,7,16,19,1,53,68,82,55,82,54,35,20,57,0,4,32,19,8,14,13,0,0,2,0,8,0,0,3,0,0,2,29,5,5,6,8,5,5,2,2,12,7,10,2,4,2,2,4,71,17,0,154,2,3,16,25,8,32,14,12,21,3,16,14,0,0,0,15,28,1,29,16,43,2,1,4,0,10,0,2,7,9,0,3,0,0,0,0,0,3,72,92,0,0,8,99,68,20,13,44,0,2,3,4,8,92,0,0,97,0,2,1,0,0,99,0,99,99,99,10,4,10,6,23,59,5,10,106.0,6,4,10.0,20.0,20.0,9,17.666667,66224,1,1,2,13,2,1,89,1,16,2,89,3,91,10,95,4,89,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
59597,0,36.000000,2,9,0,0,24,52,15,4,0,1,3,929,243,372,99,0,0,47,53,96,0,1,1,2,0,0,0,

# Lab random forest classification

### use random forest importance and selectfrommodel to find import features

In [12]:

model= SelectFromModel(RandomForestClassifier(n_estimators=100))
#SelectFromModel will select those features which importance is greater than the mean importance of all the features by default

model.fit(X_train,y_train)

model.get_support() #returns boolean true feature importance is greater than mean importance




array([False,  True, False, False,  True, False,  True,  True,  True,
        True,  True, False, False,  True,  True,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True, False, False, False,  True, False, False,  True,
        True,  True,  True,  True, False,  True,  True, False, False,
       False, False,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False, False,  True,  True, False,  True,  True,  True, False,
       False, False, False,  True,  True,  True,  True, False, False,
        True,  True,  True, False, False, False, False, False,  True,
       False, False,

In [13]:
selected_feat= X_train.columns[(model.get_support())]
len(selected_feat)
#we selected 198 columns which importance is greater than the mean impportance of all features


192

In [14]:
#get the selected features
selected_feat= X_train.columns[(model.get_support())]
len(selected_feat)

192

In [15]:
selected_feat

Index(['AGE', 'HIT', 'MALEVET', 'VIETVETS', 'WWIIVETS', 'LOCALGOV', 'STATEGOV',
       'POP901', 'POP902', 'POP903',
       ...
       'CLUSTER2', 'CLUSTER', 'ODATEW_YR', 'DOB_YR', 'MINRDATE_YR',
       'MINRDATE_MM', 'MAXRDATE_MM', 'LASTDATE_MM', 'FIRSTDATE_YR',
       'FIRSTDATE_MM'],
      dtype='object', length=192)

In [16]:
#update our selecte data based on the filtered features
X_train2=X_train[selected_feat].copy()
X_train2.shape

(7664, 192)

In [17]:

#to keep the model simple
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


kbest = SelectKBest(chi2, k=50).fit(X_train2, y_train)# k10 means we want 10 best features

df = pd.DataFrame(data = kbest.scores_, columns = ['kbest'])
df['Column'] = X_train2.columns
df_sort=df.sort_values(by = ['kbest'], ascending = False)
df_sort.head(50)


,kbest,Column
180,161398.991205,CONTROLN
76,47061.575327,IC5
7,36800.055273,POP901
40,16897.986809,HV2
39,16759.863186,HV1
9,9756.854869,POP903
8,8862.209588,POP902
69,8840.601107,MSA
57,1030.391676,HVP2
58,851.425618,HVP3


In [18]:
df_sort.tail(50)

,kbest,Column
121,1.664592,EIC4
115,1.631442,OCC5
107,1.497216,LFC5
13,1.492433,CHIL2
106,1.487814,LFC4
23,1.416907,CHILC3
22,1.414093,CHILC2
54,1.375655,ETHC1
18,1.193376,AGEC4
28,1.189203,HHN1


In [19]:
#lets include all that have a score larger than 200
len(df_sort.Column[df_sort['kbest']>200])

27

In [20]:
X_train_features=X_train2[df_sort.Column[df_sort['kbest']>200]].copy()
X_train_features

,CONTROLN,IC5,POP901,HV2,HV1,POP903,POP902,MSA,HVP2,HVP3,RAMNTALL,IC4,IC2,IC1,IC3,NGIFTALL,LASTGIFT,HVP4,NUMPROM,AVGGIFT,MINRAMNT,MAXRAMNT,CARDGIFT,DMA,RP2,RP1,HVP5
78204,133647,15640,799,626,621,379,228,1080.0,0,1,111.0,404,383,244,313,23,4.0,12,64,4.826087,3.0,7.0,12,757.0,7,2,83
55531,66224,27594,1253,1223,1139,471,377,2160.0,21,64,106.0,713,676,630,670,6,20.0,86,59,17.666667,10.0,20.0,4,505.0,78,61,94
59597,101774,9685,929,500,464,372,243,3760.0,0,1,148.0,285,257,164,224,18,9.0,11,46,8.222222,5.0,13.0,7,616.0,7,1,41
3223,45879,13616,4169,522,492,2246,1352,8280.0,0,1,36.0,288,244,200,243,8,5.0,7,46,4.500000,2.0,10.0,5,539.0,33,5,47
36266,159633,18996,386,1026,813,142,102,8780.0,19,37,194.0,477,422,408,424,11,18.0,54,73,17.636364,5.0,25.0,8,866.0,33,12,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65975,174360,24280,1369,1191,1163,456,399,6440.0,8,75,107.0,655,631,617,633,12,12.0,99,62,8.916667,3.0,12.0,7,820.0,93,93,99
85012,170231,11525,138,2194,1719,54,40,0.0,56,75,462.0,369,377,213,339,36,6.0,94,80,12.833333,5.0,25.0,18,807.0,31,15,99
38081,72452,20169,1139,1005,951,400,330,440.0,6,42,60.0,648,611,601,639,18,5.0,84,55,3.333333,2.0,5.0,6,505.0,89,56,98
59824,113329,9024,324,507,486,124,87,5880.0,0,1,220.0,257,224,214,269,10,40.0,7,69,22.000000,10.0,40.0,7,650.0,29,12,46


In [21]:
# these are the columns I am gonna use for the model

In [22]:
#select also xtest columns
X_test2=X_test[df_sort.Column[df_sort['kbest']>200]].copy()
X_test2.shape


(19082, 27)

## apply randomforest algorithm

In [23]:
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)

clf.fit(X_train_features, y_train)
print(clf.score(X_train_features, y_train))
print(clf.score(X_test2, y_test))

y_pred = clf.predict(X_test2)

display(confusion_matrix(y_test, y_pred))

0.5998173277661796
0.5948537889110156


array([[10771,  7300],
       [  431,   580]])

In [24]:
display(y_test.value_counts()) #testd ata is of course highly imbalanced

0    18071
1     1011
Name: TARGET_B, dtype: int64

In [25]:

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [26]:
model = RandomForestClassifier()
random_search = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter=25, cv = 2, n_jobs = -1)

In [27]:
#random_search.fit(X_train_features,y_train)

In [28]:
#random_search.best_params_

In [29]:
#random_search.best_score_

In [30]:
#train new model with opitmized hyperparameters to check

In [31]:
clf = RandomForestClassifier(n_estimators=1200,
                                max_depth=10,
                             min_samples_split=10,
                             min_samples_leaf =2,
                             max_features='auto',
                             max_samples=0.2,
                             bootstrap=True,
                             random_state = 42)

clf.fit(X_train_features, y_train)
print(clf.score(X_train_features, y_train))
print(clf.score(X_test2, y_test))

y_pred = clf.predict(X_test2)

display(confusion_matrix(y_test, y_pred))

0.7098121085594989
0.5902945183942983


array([[10691,  7380],
       [  438,   573]])

In [32]:
#reduced a bit false positive and false negative once

"Discuss the output and its impact in the bussiness scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the bussiness?"

false Postive: means we sent out post to people who than dont donate
false negative: we dont sent out post to people who would donate

if the cost of sending out all the post also to people who dont donate is alrger than the incoming donations, than we need to be more careful with the false positive
However it would be much more damaging the ignore sneding post to people who would donate (so the false negative)
this current model using over or underfitting reduces the data imbalance and therefore reduced false negative

so the costs of false negative can be much higher since we loose important donations,
we could improve teh data and model to furhter increase the bias towards the other side, however we would also get more false positive

# Lab 2: Linear regression model

"subset those that made a donation and use that subset to create a model to predict how much money will they give"

### # first step: feature selection

In [91]:
y_fit=X['TARGET_D']
X_fit=X.drop('TARGET_D', axis=1)

In [93]:
#feature selection with recursive feature elimination (RFE)
from sklearn.feature_selection import RFE
from sklearn import linear_model

#lets say get 20 best fetaures forthe model
lm = linear_model.LinearRegression()
rfe = RFE(lm, n_features_to_select=20, verbose=False)


rfe.fit(X_fit,y_fit)#(X_train, y_train)


RFE(estimator=LinearRegression(), n_features_to_select=20, verbose=False)

In [98]:
df_RFE = pd.DataFrame(data = rfe.ranking_, columns=['Rank'])
df_RFE['Column_name'] = pd.DataFrame(X_fit).columns
df_RFE1=df_RFE[df_RFE['Rank']==1]
df_RFE1

,Rank,Column_name
169,1,TPE3
170,1,TPE4
171,1,TPE5
172,1,TPE6
327,1,LASTDATE_YR
331,1,0
332,1,1
333,1,2
334,1,3
335,1,4


In [99]:
df_RFE1.Column_name

169           TPE3
170           TPE4
171           TPE5
172           TPE6
327    LASTDATE_YR
331              0
332              1
333              2
334              3
335              4
336              5
337              6
338              7
339              8
340              9
341             10
345             14
346             15
347             16
352             21
Name: Column_name, dtype: object

In [100]:
#only take those selected columns with rank 1, column_name with number is from one_hot encoding
X_train_regression=X_train[df_RFE1.Column_name].copy()

X_test_regression=X_test[df_RFE1.Column_name].copy()



In [101]:
# train linear regression on test set but only on people who did actually donate
#use for now same features as in earlymodel, not optimal should be changed
donated_train=pd.concat([X_train_regression, y_train,y_train_regression],axis=1)
#donated_train
donated_train_yes=donated_train[donated_train['TARGET_B']==1]
donated_train_yes.shape

(3832, 22)

In [102]:
# do x-y split again
donated_train_yes=donated_train_yes.drop(['TARGET_B'], axis = 1)# we dont need this target anymore
y_train_donated=donated_train_yes['TARGET_D']
donated_train_yes=donated_train_yes.drop(['TARGET_D'], axis = 1)


In [103]:
donated_train_yes

,TPE3,TPE4,TPE5,TPE6,LASTDATE_YR,0,1,2,3,4,5,6,7,8,9,10,14,15,16,21
78204,2,2,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
59597,6,6,0,0,95,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7154,0,0,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
18140,15,15,0,0,95,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
94083,9,9,0,0,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69577,0,0,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
65975,2,2,0,0,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
85012,0,0,0,0,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
38081,0,0,0,0,96,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
y_train_donated

78204     5.0
59597     8.0
7154     10.0
18140    15.0
94083    20.0
         ... 
69577    15.0
65975    11.0
85012    13.0
38081     4.0
34902    13.0
Name: TARGET_D, Length: 3832, dtype: float64

In [105]:
lm = linear_model.LinearRegression()
lm.fit(donated_train_yes,y_train_donated)#fit regression model on train data

LinearRegression()

In [106]:
X_test.shape # we predict that 8265 out of our 19082 people are going to donate (real postive and false positive)

(19082, 355)

In [107]:
#get data of people in the test set where we predicted that they donate

#concat X_test + y(predicts if donate frompreviousmodel) + amount (from test dataset)
Test_donated=pd.concat([X_test_regression,y_test_regression],axis=1)
Test_donated['Target_B']=y_pred
Test_donated

,TPE3,TPE4,TPE5,TPE6,LASTDATE_YR,0,1,2,3,4,5,6,7,8,9,10,14,15,16,21,TARGET_D,Target_B
65275,0,0,0,0,97,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
9484,3,3,0,0,95,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
13395,2,2,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
1466,0,0,0,0,96,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
2076,1,1,0,0,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38263,1,0,0,1,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
57783,0,0,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
26449,0,0,0,0,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0
1969,0,0,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0


In [108]:
#filter those out which we predicted donated
X_test_predict_donated=Test_donated[Test_donated['Target_B']==1]
X_test_predict_donated
#of coruse there are a lot false positive

,TPE3,TPE4,TPE5,TPE6,LASTDATE_YR,0,1,2,3,4,5,6,7,8,9,10,14,15,16,21,TARGET_D,Target_B
1466,0,0,0,0,96,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
4770,1,1,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1
92256,2,2,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
51036,0,0,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
78805,4,4,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67049,0,0,0,0,96,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1
48254,0,0,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1
8471,4,4,0,0,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
45286,3,3,0,0,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1


In [109]:
#drop targets again
# do x-y split again
X_test_reg=X_test_predict_donated.drop(['Target_B'], axis = 1)# we dont need this target anymore
y_test_donated=X_test_predict_donated['TARGET_D']
X_test_reg=X_test_reg.drop(['TARGET_D'], axis = 1)
X_test_reg

,TPE3,TPE4,TPE5,TPE6,LASTDATE_YR,0,1,2,3,4,5,6,7,8,9,10,14,15,16,21
1466,0,0,0,0,96,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4770,1,1,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
92256,2,2,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
51036,0,0,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
78805,4,4,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67049,0,0,0,0,96,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
48254,0,0,0,0,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
8471,4,4,0,0,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45286,3,3,0,0,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [111]:
predictions= lm.predict(donated_train_yes)
print('R2 train=',r2_score(y_train_donated, predictions))


predictions= lm.predict(X_test_reg)
print('R2=',r2_score(y_test_donated, predictions))

mse=np.sqrt(mean_squared_error(y_test_donated,predictions))
print('MSE=',mse)

rmse = math.sqrt(mse)
print('RMSE=',rmse)

mae = mean_absolute_error(y_test_donated, predictions)
print('MAE=', mae)

R2 train= 0.3655188036048329
R2= -13.494715634164102
MSE= 13.83023856406869
RMSE= 3.718902871018372
MAE= 12.293276454445227


In [ ]:
# the feature selection might have resulted in the bad scores

#I tried different feature selections (and delted them again), i dont know whats wrong


#already the train set is not well predicted, but the test set is even minus
# i dont know how to meaningful select the features


#so right now the results are much worse to a naive scenario